<a href="https://colab.research.google.com/github/tohidhaghighy/FaketextCreator/blob/main/Fakenews_makenews_Albert_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import Dense
from tqdm import tqdm
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 249 kB 5.1 MB/s 
     |████████████████████████████████| 6.6 MB 59.5 MB/s 
     |████████████████████████████████| 1.3 MB 59.7 MB/s 
     |████████████████████████████████| 9.1 MB 59.1 MB/s 
     |████████████████████████████████| 365 kB 70.2 MB/s 
     |████████████████████████████████| 4.7 MB 40.3 MB/s 
     |████████████████████████████████| 43 kB 2.5 MB/s 
     |████████████████████████████████| 1.8 MB 49.8 MB/s 
     |████████████████████████████████| 596 kB 73.3 MB/s 
     |████████████████████████████████| 101 kB 12.9 MB/s 
     |████████████████████████████████| 157 kB 70.5 MB/s 
     |████████████████████████████████| 181 kB 75.4 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 157 kB 68.9 MB/s 
     |████████████████████████████████| 157 kB 70.3 MB/s 
     |██████████████████████████████

In [9]:
stop_words = ["A","a","i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "'s", "'t","s","t","d","r","u", "can", "will", "just", "don", "should", "now","}","{",")","(","0","1","2","3","4","5","6","7","8","9","'",":",",","@","as","for","]","[","?","."]

# Fakenews Creator
- This is create by tohid haghighi 
- For final proposal

In [ ]:
import typing

In [ ]:
!python --version

Python 3.7.13


In [11]:
from tqdm import tqdm
from nltk.stem.porter import PorterStemmer

#------------------------------- Albert model ------------------------
from transformers import AutoTokenizer, AutoModelForMaskedLM, FillMaskPipeline
tokenizer = AutoTokenizer.from_pretrained("Jawaher/LIAR-fake-news-roberta-base")
model_roberta = AutoModelForMaskedLM.from_pretrained("Jawaher/LIAR-fake-news-roberta-base")
model_roberta.eval()
#---------------------------------------------------------------------

def MakeMaskData_Albert(text):
  '''
  text = Sentence that must create fake text with that
  whichSentence = Index of which sentence that can change
  '''
  ps = PorterStemmer()
  maskedList=[]
  words=text.split(' ')
  words = [word for word in words if not word in stop_words]
  for item in tqdm(words):
    pipeline = FillMaskPipeline(model_roberta, tokenizer)
    originalText = text
    originalText=originalText.replace(item,'<mask>',1)
    res_hf = pipeline(originalText)
    for mask in res_hf:
      originalText = text
      originalText=originalText.replace(item,mask['token_str'])
      maskedList.append(originalText)
  return maskedList
    

In [12]:
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='openai-gpt')
from tqdm import tqdm
def MakeMaskeData_GPT(text,whichSentence:int = 0):
  '''
  text = Sentence that must create fake text with that
  whichSentence = Index of which sentence that can change
  '''
  listofsentences = text.split('.')
  set_seed(32)
  generatedList=generator(listofsentences[whichSentence], max_length=120, num_return_sequences=1)
  originalText = text
  originalText=originalText.replace(listofsentences[whichSentence],generatedList[0],1)
  return generatedList[0]

Some weights of OpenAIGPTLMHeadModel were not initialized from the model checkpoint at openai-gpt and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
def FaketextCreator(text:str ,algorithm:str = "Albert" ,replace:bool = True ,placeofword:str="Inplace" ,removeStepWords:bool = True ,k: int = 3):
  '''
  text = sentence that must create fake text with that
  algorithm = we have 2 algorithm fisrt one is "Albert" that can change words in sentence with mask base second one is "GPT"
  replace = Do you want to replace new word or sentence or add before that text
  placeofword = we have 3 place for word left or right or inplace
  k = Count of word or sentence we must change
  '''
  if(algorithm=="Albert"):
    return MakeMaskData_Albert(text)
  elif(algorithm=="GPT"):
    return MakeMaskeData_GPT(text)
  return ""

In [7]:
print(FaketextCreator('this is for me'))

['this', 'is', 'for', 'me']
